In [22]:
from utils.find_miner_id import ReadTokens
import json
from datetime import datetime


In [32]:
def calculate_pplns_participation(shares_data, block_data, n_factor):
    # Sort shares by blockheight in descending order
    shares_data.sort(key=lambda x: x['blockheight'], reverse=True)
    block_height = block_data['blockheight']
    network_difficulty = block_data['networkdifficulty']
    
    target_shares = network_difficulty * n_factor
    print(network_difficulty, 'net diff', target_shares, 'target shares')

    # Initialize variables
    total_shares = 0
    valid_shares = []
    miner_shares = {}

    # Collect shares until we reach or exceed the target
    for share in shares_data:
        
        if share['blockheight'] > block_height:
            continue

        valid_shares.append(share)
        total_shares += share['difficulty']

        miner = share['miner']
        if miner not in miner_shares:
            miner_shares[miner] = 0
        miner_shares[miner] += share['difficulty']

        if total_shares >= target_shares:
            print(share, 'share')
            break

    # Calculate participation percentages
    participation = {miner: shares / total_shares for miner, shares in miner_shares.items()}

    return participation, total_shares

In [24]:
reader = ReadTokens()
# Set the N factor (this could be user-defined)
n_factor = 0.5  # For example, 2 times the network difficulty


In [25]:
shares_data = reader.get_api_data('http://5.78.102.130:8000/miningcore/shares')
tshares_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/shares')

shares_data[0], tshares_data[0]

({'poolid': 'ErgoSigmanauts',
  'blockheight': 1369762,
  'difficulty': 0.010991475941720133,
  'networkdifficulty': 224470.58289740514,
  'miner': '9fBmHyFqdegV7a8z4bG9KEPXPM3JunVS52TrsQBBxQhjy4pdUq2',
  'worker': '6x_MIXED',
  'useragent': 'Rigel/1.18.2',
  'ipaddress': '::ffff:78.157.8.17',
  'source': 'ErgoSigmanauts',
  'created': '2024-10-09T10:54:57.104188+00:00'},
 {'poolid': 'Shark-Dev-Pool',
  'blockheight': 1364736,
  'difficulty': 0.010449310109866801,
  'networkdifficulty': 246939.2640119879,
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'worker': 'MT-MASSIVE',
  'useragent': 'teamredminer/0.10.21',
  'ipaddress': '::ffff:216.147.121.136',
  'source': 'Shark-Dev-Pool',
  'created': '2024-10-02T09:43:37.652364+00:00'})

In [26]:
# Load block data
block_data = reader.get_api_data('http://5.78.102.130:8000/miningcore/blocks')
tblock_data = reader.get_api_data('http://188.245.66.57:8000/miningcore/blocks')
block_data[0], tblock_data[0]

({'id': 1,
  'poolid': 'ErgoSigmanauts',
  'blockheight': 1345216,
  'networkdifficulty': 225936.58304610793,
  'status': 'confirmed',
  'type': None,
  'confirmationprogress': 1.0,
  'effort': 0.028338019989095103,
  'transactionconfirmationdata': '5003003396d03cee',
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'reward': 24.014597512,
  'source': 'ErgoSigmanauts',
  'hash': 'cac7398ad047d11b9595c10666b04f0ec1d710e1b560bc13e40e1d2c345890a5',
  'created': '2024-09-05T00:57:19.492068+00:00'},
 {'id': 2,
  'poolid': 'Shark-Dev-Pool',
  'blockheight': 1364656,
  'networkdifficulty': 246939.2640119879,
  'status': 'confirmed',
  'type': None,
  'confirmationprogress': 1.0,
  'effort': 4.846233239117975,
  'transactionconfirmationdata': 'c00a9835c0ca8e2e',
  'miner': '9ehJZvPDgvCNNd2zTQHxnSpcCAtb1kHbEN1VAgeoRD5DPVApYkk',
  'reward': 24.019210101,
  'source': 'Shark-Dev-Pool',
  'hash': '1a28de327dd4617e2b6f0640ffb5f4f1e86c711f6072359d73db283f6d56a64f',
  'created': '20

In [33]:

# Set the N factor (this could be user-defined)
n_factor = 0.5  # For example, 2 times the network difficulty

block_data.sort(key=lambda x: x['blockheight'], reverse=True)
blocks = {'blockheight': 1370030, #block_data[0]['blockheight'],
          'networkdifficulty': block_data[0]['networkdifficulty']}
    # network_difficulty = block_data[0]['networkdifficulty']
# Calculate participation
participation, total_shares = calculate_pplns_participation(shares_data, block_data, blocks, n_factor)

# Print results
print(f"Participation for block {blocks['blockheight']} (N factor: {n_factor}):")
print('TOTAL SHARES: {}'.format(total_shares))
for miner, percentage in participation.items():
    print(f"Miner {miner}: {percentage:.2%}")

# Optional: Calculate rewards
block_reward = block_data[0]['reward']
print(f"\nReward distribution for block reward of {block_reward}:")
for miner, percentage in participation.items():
    miner_reward = block_reward * percentage
    print(f"Miner {miner}: {miner_reward:.8f}")

205208.88674544086 net diff 102604.44337272043 target shares
Participation for block 1369627 (N factor: 0.5):
TOTAL SHARES: 2941.0713194915375
Miner 9hwEKqgW4XjRi7uYjRcsZHiRiuNT3dMHsR3J75BNSrGDQyKuz8Q: 2.06%
Miner 9eoPRhRmJQ71TPSkk2eUmMXYpxUVvZvFQo5uB9qjPBM4e3hC82J: 1.87%
Miner 9feNRbYuVz2nQy1V8t2z5saYnWKwmGJ2nYgcMdXBjqYXqHMeWnv: 0.38%
Miner 9gNEKbpPSNfgWkhxsMF4Z9L7uNiXFfsEX4XTVF5XK4kwpQBRsEQ: 50.46%
Miner 9fLytFFzTYALknc2AZ2dRKeg8sLZwe4LX5qAB3FwDysMEeRTHkV: 1.42%
Miner 9fy4KkHt9Xavq9R7Wq16euW4JjH1mYwBtV7SNNtAtwnQ6qSZL4w: 1.03%
Miner 9fBmHyFqdegV7a8z4bG9KEPXPM3JunVS52TrsQBBxQhjy4pdUq2: 4.40%
Miner 9gLMeVQbDWX522WUzBwmVnr2Avqki7wdz7c9MpcpkgVJE5DUhcN: 2.51%
Miner 9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL8ygPH: 1.33%
Miner 9fRJtg8m4Tgurq8cMSMs2m1pMk4MW3GHk8eLU6wuKUDm4edRioN: 3.99%
Miner 9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2TuYsAik: 7.03%
Miner 9fyDqkVk3iDXPNcquyLBehEtiZiqdKb27PnugemJEpGYbTJM7tN: 1.62%
Miner 9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsNLUuxo: 3.67%
Miner 9fNJd

In [34]:
participation

{'9hwEKqgW4XjRi7uYjRcsZHiRiuNT3dMHsR3J75BNSrGDQyKuz8Q': 0.020609328983880293,
 '9eoPRhRmJQ71TPSkk2eUmMXYpxUVvZvFQo5uB9qjPBM4e3hC82J': 0.018706301101889096,
 '9feNRbYuVz2nQy1V8t2z5saYnWKwmGJ2nYgcMdXBjqYXqHMeWnv': 0.0037638720643013855,
 '9gNEKbpPSNfgWkhxsMF4Z9L7uNiXFfsEX4XTVF5XK4kwpQBRsEQ': 0.5045867708436039,
 '9fLytFFzTYALknc2AZ2dRKeg8sLZwe4LX5qAB3FwDysMEeRTHkV': 0.014156952388419663,
 '9fy4KkHt9Xavq9R7Wq16euW4JjH1mYwBtV7SNNtAtwnQ6qSZL4w': 0.010298445959613415,
 '9fBmHyFqdegV7a8z4bG9KEPXPM3JunVS52TrsQBBxQhjy4pdUq2': 0.04396769087614725,
 '9gLMeVQbDWX522WUzBwmVnr2Avqki7wdz7c9MpcpkgVJE5DUhcN': 0.0251359975888668,
 '9hSk13XDU8keWAp3KxKSgUVCkfDcw577jAjM6SgbRpuYeL8ygPH': 0.013336309542690107,
 '9fRJtg8m4Tgurq8cMSMs2m1pMk4MW3GHk8eLU6wuKUDm4edRioN': 0.03987307996086507,
 '9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2TuYsAik': 0.07034903678605704,
 '9fyDqkVk3iDXPNcquyLBehEtiZiqdKb27PnugemJEpGYbTJM7tN': 0.016233693667686414,
 '9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsNLUuxo': 0.036665608883

In [16]:
a = 77.8
b = 2.12
c = 21.56

total_shares = a + b + c

In [17]:
a / total_shares, b / total_shares, c / total_shares

(0.7666535277887268, 0.020890815924320065, 0.21245565628695307)